In [4]:
import pandas as pd
import gcsfs as gcs
from pyspark import SparkContext
from pyspark.sql import SQLContext

In [5]:
#data = pd.read_csv('s3://zengageprodadminapp/crimes_sample.csv')
data = pd.read_csv('gs://bigdatahw2/crimes_sample.csv')

In [6]:
sc = SparkContext('local','test')
sqlcont = SQLContext(sc)

In [7]:
data_sc = sqlcont.createDataFrame(data)

In [8]:
data_sc.describe()

DataFrame[summary: string, ID: string, Case Number: string, Date: string, Block: string, IUCR: string, Primary Type: string, Description: string, Location Description: string, Beat: string, District: string, Ward: string, Community Area: string, FBI Code: string, X Coordinate: string, Y Coordinate: string, Year: string, Updated On: string, Latitude: string, Longitude: string, Location: string]

In [9]:
data.shape

(9999, 22)

In [10]:
data_2007 = data[data['Year']==2007]

In [11]:
data_2007.describe()

,ID,Beat,District,Ward,Community Area,X Coordinate,Y Coordinate,Year,Latitude,Longitude
count,4.637000e+03,4637.000000,4637.000000,4637.000000,4635.00000,4.637000e+03,4.637000e+03,4637.0,4637.000000,4637.000000
mean,5.995343e+06,1105.449429,10.471641,21.516066,39.84315,1.164196e+06,1.882003e+06,2007.0,41.831822,-87.673090
std,5.830378e+03,652.100461,6.392732,13.099573,21.24276,1.629498e+04,3.034635e+04,0.0,0.083463,0.059326
min,5.988306e+06,111.000000,1.000000,1.000000,0.00000,1.098012e+06,1.813910e+06,2007.0,41.644608,-87.915105
25%,5.990500e+06,623.000000,6.000000,9.000000,25.00000,1.151645e+06,1.855965e+06,2007.0,41.760100,-87.718588
50%,5.993468e+06,932.000000,9.000000,21.000000,38.00000,1.166063e+06,1.879297e+06,2007.0,41.824550,-87.665895
75%,5.999302e+06,1533.000000,15.000000,30.000000,61.00000,1.176685e+06,1.905309e+06,2007.0,41.895878,-87.627512
max,6.009574e+06,2535.000000,25.000000,50.000000,77.00000,1.203514e+06,1.950679e+06,2007.0,42.020285,-87.530509


In [12]:
crime_table_data = data_2007.groupby(['Primary Type']).size()
#crime_table_data.to_sql('crimestable', conn, schema=None, if_exists='fail', index=True, index_label=None, chunksize=None, dtype=None)

In [13]:
import mysql.connector

In [14]:
mydb = mysql.connector.connect(
  host="35.192.215.142",
  user="root",
  passwd="XXXXXX",
    database="crimedb"
)

In [15]:
mycursor = mydb.cursor()
mycursor.execute("CREATE TABLE crimestable (crime VARCHAR(255), count int)")

ProgrammingError: 1050 (42S01): Table 'crimestable' already exists

In [20]:
for crime,count in zip (df_crime_table['Primary Type'], df_crime_table[0]):
    sql = "INSERT INTO crimestable (crime, count) VALUES " + "(" +  crime + "," + str(count) + ")"
    print (sql)


INSERT INTO crimestable (crime, count) VALUES (ARSON,4)
INSERT INTO crimestable (crime, count) VALUES (ASSAULT,225)
INSERT INTO crimestable (crime, count) VALUES (BATTERY,793)
INSERT INTO crimestable (crime, count) VALUES (BURGLARY,230)
INSERT INTO crimestable (crime, count) VALUES (CRIM SEXUAL ASSAULT,14)
INSERT INTO crimestable (crime, count) VALUES (CRIMINAL DAMAGE,522)
INSERT INTO crimestable (crime, count) VALUES (CRIMINAL TRESPASS,119)
INSERT INTO crimestable (crime, count) VALUES (DECEPTIVE PRACTICE,215)
INSERT INTO crimestable (crime, count) VALUES (GAMBLING,7)
INSERT INTO crimestable (crime, count) VALUES (INTERFERENCE WITH PUBLIC OFFICER,8)
INSERT INTO crimestable (crime, count) VALUES (KIDNAPPING,1)
INSERT INTO crimestable (crime, count) VALUES (LIQUOR LAW VIOLATION,11)
INSERT INTO crimestable (crime, count) VALUES (MOTOR VEHICLE THEFT,186)
INSERT INTO crimestable (crime, count) VALUES (NARCOTICS,732)
INSERT INTO crimestable (crime, count) VALUES (OFFENSE INVOLVING CHILDREN,

In [18]:
df_crime_table = crime_table_data.to_frame().reset_index()

In [19]:
for crime,count in zip (df_crime_table['Primary Type'], df_crime_table[0]):
    sql = "INSERT INTO crimestable (crime, count) VALUES (%s, %s)"
    val = (crime, count)
    mycursor.execute(sql, val)

mydb.commit()